![Imgur](https://i.imgur.com/5pXzCIu.png)

# Data Science va Sun'iy Intellekt Praktikum

## 5-MODUL. Machine Learning

### Portfolio uchun vazifa: Toshkent shahrida uylarning narxini aniqlash.

Ushbu amaliyotda sizning vazifangiz berilgan ma`lumotlar asosida Toshkent shahridagi uylarning narxini aniqlash.

In [94]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

df = pd.read_csv('https://raw.githubusercontent.com/anvarnarz/praktikum_datasets/main/housing_data_08-02-2021.csv', index_col=0)
df.head()

,district,rooms,size,level,max_levels,price
location,,,,,,
"город Ташкент, Юнусабадский район, Юнусабад 8-й квартал",Юнусабадский,3,57,4,4,52000
"город Ташкент, Яккасарайский район, 1-й тупик Шота Руставели",Яккасарайский,2,52,4,5,56000
"город Ташкент, Чиланзарский район, Чиланзар 2-й квартал",Чиланзарский,2,42,4,4,37000
"город Ташкент, Чиланзарский район, Чиланзар 9-й квартал",Чиланзарский,3,65,1,4,49500
"город Ташкент, Чиланзарский район, площадь Актепа",Чиланзарский,3,70,3,5,55000


# Ustunlar ta'rifi
- `location` - sotilayotgan uy manzili
- `district` - uy joylashgan tuman
- `rooms` - xonalar soni
- `size` - uy maydoni (kv.m)
- `level` - uy joylashgan qavat
- `max_levels` - ja'mi qavatlar soni
- `price` - uy narxi

## Vazifani CRSIP-DM Metolodgiyasi yordamida bajaring.
<img src="https://i.imgur.com/dzZnnYi.png" alt="CRISP-DM" width="800"/>

In [95]:
df.set_index('rooms',inplace=True)

In [96]:
df = df[df['size'] != 'Площадьземли:1сот']
df = df[df['price'] != 'Договорная']
df['price'] = pd.to_numeric(df['price'])
df['size']=pd.to_numeric(df['size'])

df['price']= df['price'] < 1000000

In [96]:
# median_price = np.median(df['price'])
# df['price'] = df['price'].replace('Договорная', median_price)

In [97]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7465 entries, 3 to 1
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   district    7465 non-null   object 
 1   size        7465 non-null   float64
 2   level       7465 non-null   int64  
 3   max_levels  7465 non-null   int64  
 4   price       7465 non-null   bool   
dtypes: bool(1), float64(1), int64(2), object(1)
memory usage: 298.9+ KB


In [98]:
# train test setga ajratish
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)

X_train = train_set.drop("price", axis=1)
y = train_set["price"].copy()

X_num = X_train.drop("district", axis=1)

In [99]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# sonlar bilan ishlovchi pipeline

num_pipeline = Pipeline([
          ('std_scaler', StandardScaler())
])


In [100]:
from sklearn.compose import ColumnTransformer

num_attribs = list(X_num)
cat_attribs = ['district']

full_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_attribs),
    ('cat', OneHotEncoder(), cat_attribs)
])

X_prepared = full_pipeline.fit_transform(X_train)

In [101]:
from sklearn.linear_model import LinearRegression

LR_model = LinearRegression()

LR_model.fit(X_prepared, y)

LinearRegression()

In [102]:
X_test = test_set.drop('price', axis=1)
y_test = test_set['price'].copy()

X_test_prepared = full_pipeline.transform(X_test)

y_predicted = LR_model.predict(X_test_prepared)

In [103]:
from sklearn.metrics import mean_squared_error
lin_mse = mean_squared_error(y_test, y_predicted)
# RMSE hisoblaymiz
lin_rmse = np.sqrt(lin_mse)
print(lin_rmse)

0.05172152514017562


In [104]:
from sklearn.ensemble import RandomForestRegressor
RF_model = RandomForestRegressor()
RF_model.fit(X_prepared, y)

RandomForestRegressor()

In [105]:
y_predicted = RF_model.predict(X_test_prepared)
lin_mse = mean_squared_error(y_test, y_predicted)
# RMSE hisoblaymiz
lin_rmse = np.sqrt(lin_mse)
print(lin_rmse)

0.051881128226948424


In [112]:
import joblib

filename = 'LR_model.jbl' # faylga istalgan nom beramiz
joblib.dump(LR_model, filename)

['LR_model.jbl']